In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
#https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/datasets/uktradecountrybycommodityimports

url = "https://www.ons.gov.uk/file?uri=%2feconomy%2fnationalaccounts%2fbalanceofpayments%2fdatasets%2fuktradecountrybycommodityimports%2fcurrent/cxcimportshistoricaldata.zip"

In [3]:
r = requests.get(url) 

brexit_FEB21 = zf.ZipFile(io.BytesIO(r.content))

brexit_FEB21.extractall(".\\data")

filename = brexit_FEB21.namelist()

In [4]:
df = pd.read_excel(".\\data" + "\\" + filename[0], sheet_name= "Country by Commodity Import")

In [5]:
df.head()

,COMMODITY,COUNTRY,DIRECTION,1997JAN,1997FEB,1997MAR,1997APR,1997MAY,1997JUN,1997JUL,...,2017MAR,2017APR,2017MAY,2017JUN,2017JUL,2017AUG,2017SEP,2017OCT,2017NOV,2017DEC
0,0 Food & live animals,AD Andorra,IM Imports,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0 Food & live animals,AE United Arab Emirates,IM Imports,0.27,0.07,2.52,22.38,5.06,0.23,0.62,...,1.12,0.57,0.84,0.88,0.96,0.98,0.64,1.31,1.06,0.55
2,0 Food & live animals,AF Afghanistan,IM Imports,0.06,0.07,0.00,0.23,0.05,0.05,0.00,...,0.00,0.00,0.00,0.04,0.02,0.00,0.00,0.03,0.00,0.02
3,0 Food & live animals,AG Antigua and Barbuda,IM Imports,0.02,0.12,0.00,0.00,0.04,0.01,0.00,...,0.00,0.00,0.06,0.00,0.01,0.01,0.00,0.01,2.71,0.02
4,0 Food & live animals,AI Anguilla,IM Imports,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [6]:
df = df.melt(id_vars = ["COMMODITY", "COUNTRY", "DIRECTION"])

df.drop("DIRECTION", axis = 1, inplace = True)

df.variable= pd.to_datetime(df.variable, format="%Y%b")

df.rename({"variable":"time", "value": "imports"},inplace = True, axis = 1)

df.COUNTRY = df.COUNTRY.str[3:]

df.imports = df.imports*1000000

In [7]:
grp = df[df.COMMODITY == 'T Total'].groupby("COUNTRY")

test = grp.agg({"imports": "sum"})

In [8]:
topcntry = test.sort_values(by = "imports", ascending = False).iloc[0:23].index.to_list()

In [9]:
df = df[df.COUNTRY.isin(topcntry)]

In [10]:
df.COMMODITY.replace({"T Total": "Total"}, inplace = True)
df.COUNTRY.replace({"United States inc Puerto Rico": "United States"}, inplace = True)
df.COUNTRY.replace({"Extra EU 28 (Rest of World)": "Excl. EU 28 (Rest of World)"}, inplace = True)

In [11]:
grp = df.groupby(["COUNTRY"])

In [12]:
def ex_metal_total(df):
        
    if not (df[df["COMMODITY"] == "9 Unspecified goods"].empty):
                
        foo = pd.DataFrame([])

        fooall = df[df["COMMODITY"] == "Total"].set_index(["time"])["imports"] 
        
        foobar = df[df["COMMODITY"] == "9 Unspecified goods"].set_index(["time"])["imports"]

        foo["imports"] = fooall - foobar
    # Grab the total and then sbutract off the unspecified goods
    
        foo["COUNTRY"] = df.COUNTRY.unique()[0]
    
        #print(df.COUNTRY.unique())
    
        foo["COMMODITY"] = "Total (ex. metals)"
        
        foo.reset_index(inplace = True)
        
        foo[df.columns]
        
        df = df.append(foo)
    
    return df

In [13]:
df = grp.apply(ex_metal_total)

In [14]:
df.drop(["COUNTRY"], axis = 1, inplace = True)

df.reset_index(inplace = True)

df.drop(["level_1"], axis = 1, inplace = True)

In [18]:
out_file = ".\\data"+ "\\UK-imports-1997-2017.parquet"

pq.write_table(pa.Table.from_pandas(df), out_file)

C:\ProgramData\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:383: FutureWarning: RangeIndex._start is deprecated and will be removed in a future version. Use RangeIndex.start instead
  'start': level._start,
C:\ProgramData\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:384: FutureWarning: RangeIndex._stop is deprecated and will be removed in a future version. Use RangeIndex.stop instead
  'stop': level._stop,
C:\ProgramData\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:385: FutureWarning: RangeIndex._step is deprecated and will be removed in a future version. Use RangeIndex.step instead
  'step': level._step


In [17]:
df.tail()

,COUNTRY,COMMODITY,time,imports
730291,Whole world,Total (ex. metals),2017-08-01,3.862600e+10
730292,Whole world,Total (ex. metals),2017-09-01,4.183200e+10
730293,Whole world,Total (ex. metals),2017-10-01,4.127100e+10
730294,Whole world,Total (ex. metals),2017-11-01,4.213700e+10
730295,Whole world,Total (ex. metals),2017-12-01,3.663500e+10
